In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# Define paths to your training, validation, and testing data
train_data_dir = '/Users/arshdeepsingh/Documents/tuberculosis_detection_v2/train'
val_data_dir = '/Users/arshdeepsingh/Documents/tuberculosis_detection_v2/validation'
test_data_dir = '/Users/arshdeepsingh/Documents/tuberculosis_detection_v2/test'

# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

val_test_datagen = ImageDataGenerator(rescale=1./255)

# Set the input image size for DenseNet model
img_width, img_height = 224, 224

# Batch size for training and validation
batch_size = 32

# Define the number of models in the ensemble
num_models = 3

# Define epochs and early stopping criteria
epochs = 10
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Function to create and compile model
def create_model():
    base_model = DenseNet121(weights='imagenet', include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Initialize list to store models
models = []

# Train multiple models with different architectures or initializations
for i in range(num_models):
    # Prepare train and validation data
    train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height), batch_size=batch_size, class_mode='binary')
    val_generator = val_test_datagen.flow_from_directory(val_data_dir, target_size=(img_width, img_height), batch_size=batch_size, class_mode='binary')

    # Create and train the model
    model = create_model()
    model.fit(train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=epochs, validation_data=val_generator, validation_steps=val_generator.samples // batch_size, callbacks=[early_stopping])
    
    # Add trained model to list
    models.append(model)

# Evaluate ensemble model on test data
test_generator = val_test_datagen.flow_from_directory(test_data_dir, target_size=(img_width, img_height), batch_size=batch_size, class_mode='binary')
predictions = np.mean([model.predict(test_generator) for model in models], axis=0)
predictions_binary = np.round(predictions)
y_true = test_generator.classes

# Generate classification report and confusion matrix for ensemble predictions
print(classification_report(y_true, predictions_binary))
print(confusion_matrix(y_true, predictions_binary))


Found 2940 images belonging to 2 classes.
Found 633 images belonging to 2 classes.


Epoch 1/10
91/91 [==============================] - 319s 3s/step - loss: 0.1513 - accuracy: 0.9519 - val_loss: 1.5767 - val_accuracy: 0.9062
Epoch 2/10
91/91 [==============================] - 315s 3s/step - loss: 0.0840 - accuracy: 0.9725 - val_loss: 0.7959 - val_accuracy: 0.7911
Epoch 3/10
91/91 [==============================] - 312s 3s/step - loss: 0.0764 - accuracy: 0.9739 - val_loss: 1.1979 - val_accuracy: 0.8816
Epoch 4/10
91/91 [==============================] - 315s 3s/step - loss: 0.0599 - accuracy: 0.9852 - val_loss: 1.9868 - val_accuracy: 0.4211
Epoch 5/10
91/91 [==============================] - 313s 3s/step - loss: 0.0623 - accuracy: 0.9804 - val_loss: 0.2267 - val_accuracy: 0.9013
Epoch 6/10
91/91 [==============================] - 312s 3s/step - loss: 0.0656 - accuracy: 0.9766 - val_loss: 5.0699 - val_accuracy: 0.1793
Epoch 7/10
91/91 [==============================] - 322s 4s/step - loss: 0.0635 - accuracy: 0.9783 - val_loss: 0.0483 - val_accuracy: 0.9934
Epoch 8/10
91

Epoch 1/10
91/91 [==============================] - 313s 3s/step - loss: 0.1456 - accuracy: 0.9529 - val_loss: 18.2538 - val_accuracy: 0.1760
Epoch 2/10
91/91 [==============================] - 308s 3s/step - loss: 0.0804 - accuracy: 0.9694 - val_loss: 1.4138 - val_accuracy: 0.8882
Epoch 3/10
91/91 [==============================] - 310s 3s/step - loss: 0.0972 - accuracy: 0.9684 - val_loss: 0.7601 - val_accuracy: 0.8569
Epoch 4/10
91/91 [==============================] - 310s 3s/step - loss: 0.0672 - accuracy: 0.9790 - val_loss: 0.9132 - val_accuracy: 0.9030
Epoch 5/10
91/91 [==============================] - 306s 3s/step - loss: 0.0474 - accuracy: 0.9869 - val_loss: 0.1094 - val_accuracy: 0.9786
Epoch 6/10
91/91 [==============================] - 309s 3s/step - loss: 0.0874 - accuracy: 0.9708 - val_loss: 0.1677 - val_accuracy: 0.9178
Epoch 7/10
91/91 [==============================] - 307s 3s/step - loss: 0.0591 - accuracy: 0.9807 - val_loss: 1.6752 - val_accuracy: 0.6414
Epoch 8/10
9

Epoch 1/10
91/91 [==============================] - 313s 3s/step - loss: 0.1438 - accuracy: 0.9529 - val_loss: 4.2025 - val_accuracy: 0.5049
Epoch 2/10
91/91 [==============================] - 310s 3s/step - loss: 0.0809 - accuracy: 0.9697 - val_loss: 1.3987 - val_accuracy: 0.7237
Epoch 3/10
91/91 [==============================] - 307s 3s/step - loss: 0.0724 - accuracy: 0.9763 - val_loss: 0.1090 - val_accuracy: 0.9572
Epoch 4/10
91/91 [==============================] - 305s 3s/step - loss: 0.0450 - accuracy: 0.9852 - val_loss: 0.4236 - val_accuracy: 0.8898
Epoch 5/10
91/91 [==============================] - 305s 3s/step - loss: 0.0519 - accuracy: 0.9849 - val_loss: 0.7802 - val_accuracy: 0.9030
Epoch 6/10
91/91 [==============================] - 306s 3s/step - loss: 0.0626 - accuracy: 0.9780 - val_loss: 1.6509 - val_accuracy: 0.8766
Found 630 images belonging to 2 classes.
20/20 [==============================] - 17s 773ms/step
              precision    recall  f1-score   support

  

In [11]:
model.save('tb3_model.h5')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_generator)

# Print the test loss and accuracy
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


20/20 [==============================] - 16s 777ms/step - loss: 0.1230 - accuracy: 0.9492
Test Loss: 0.12301963567733765
Test Accuracy: 0.9492063522338867


In [13]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(val_generator)

# Print validation loss and accuracy
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)


20/20 [==============================] - 16s 776ms/step - loss: 0.1110 - accuracy: 0.9542
Validation Loss: 0.11097970604896545
Validation Accuracy: 0.9541864395141602
